### BIOSTAT 257: HW3

#### Q1: Formula

In [1]:
# load libraries
using BenchmarkTools, DelimitedFiles, Images, LinearAlgebra, Random

Consider a linear mixed effects model,
        $$\mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n $$
 where,
 - $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,
 - $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,
 - $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individuaL,
 - $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$, 
 - $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and
 - $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

#### Q1: Formula

Write down the log-likelihood of the  𝑖 -th datum  (𝐲𝑖,𝐗𝑖,𝐙𝑖)  given parameters  (𝜷,𝚺,𝜎2).

Start by finding the distribution of $Y_i$: 

- $E(Y_i) = X_i\beta$ 
- $Var(Y_i) = Z_i\Sigma Z_i^{T} + \sigma^2 I$
- Therefore, $Y_i \sim MVN(X_i\beta, Z_i\Sigma Z_i^{T} + \sigma^2 I)$

We use the pdf of the multivariate normal distribution with the above parameters to derive the log-likelihood function. Below is the derivation:

\begin{align}
\frac{n_i}{2} \log(2\pi) - \frac{1}{2} \log \vert Z_i\Sigma Z_i^{T} + \sigma^2 I \vert - \frac{1}{2}(Y_i -X_i \beta)^{T}(Z_i\Sigma Z_i^{T} + \sigma^2 I)^{-1}(Y_i - X_i \beta)
\end{align}

We can use a Cholesky Decomposition to rewrite $\Sigma$ as $LL'$ where $L$ is a lower triangular matrix.

This gives us:

\begin{align}
\frac{n_i}{2} \log(2\pi) - \frac{1}{2} \log \vert Z_i LL^{T} Z_i^{T} + \sigma^2 I \vert - \frac{1}{2}(Y_i -X_i \beta)^{T}(Z_i\ LL^{T} Z_i^{T} + \sigma^2 I)^{-1}(Y_i - X_i \beta)
\end{align}

Now let $Z_iL = M$. This gives us:

\begin{align}
\frac{n_i}{2} \log(2\pi) - \frac{1}{2} \log \vert MM^{T} + \sigma^2 I \vert - \frac{1}{2}(Y_i -X_i \beta)^{T}(MM^{T} + \sigma^2 I)^{-1}(Y_i - X_i \beta)
\end{align}

We can take advantage of the easy + low rank properties to evaluate $(MM^{T} + \sigma^2 I)^{-1}$ using the Sherman-Woodbury formula, and $\vert MM^{T} + \sigma^2 I \vert$ using the matrix determinant lemma.

Starting with the Sherman-Woodbury formula, we evaluate $(MM^{T} + \sigma^2 I)^{-1}$ to be:

\begin{align}
(MM^{T} + \sigma^2 I)^{-1} 
&= \frac{1}{\sigma^2}I - \frac{1}{\sigma^4} M\big(I + \frac{1}{\sigma^2}M^{T}M\big)^{-1}M^{T}
\end{align}

It can be proved that $\big(I + \frac{1}{\sigma^2}M^{T}M\big)$ is positive semi-definite, and if we assume M is full rank, then it is positive-definite, which would allow us to apply another Cholesky Decomposition to this matrix. 
Let this Cholesky Decomposition be: $\big(I + \frac{1}{\sigma^2}M^{T}M\big) = AA^{T}$, where $A$ is a lower triangular matrix. This gives us:
\begin{align}
(MM^{T} + \sigma^2 I)^{-1} 
&= \frac{1}{\sigma^2}I - \frac{1}{\sigma^4} M\big(I + \frac{1}{\sigma^2}M^{T}M\big)^{-1}M^{T} \\
&= \frac{1}{\sigma^2}I - \frac{1}{\sigma^4} M(AA^{T})^{-1}M^{T}
\end{align}

Next we move onto the matrix determinant lemma and we evaluate $\vert MM^{T} + \sigma^2 I \vert$:
\begin{align}
\vert MM^{T} + \sigma^2 I \vert &= \sigma^{2n_i} \vert \big(I + \frac{1}{\sigma^2}M^{T}M\big) \vert \\
&=  \sigma^{2n_i} \vert AA^{T} \vert
\end{align}

Putting these together, we evaluate the log-likelihood to be:
\begin{align}
&\frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i - X_i \beta)^{T}(Y_i - X_i \beta) + \frac{1}{2 \sigma^4}(Y_i - X_i \beta)^{T}M(AA^{T})^{-1}M^{T}(Y_i - X_i \beta) \\
&= \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i^{T}Y_i - 2Y_i^{T}X_i \beta + \beta^{T}X_i^{T}X_i \beta) + \frac{1}{2 \sigma^4}(Y_i - X_i \beta)^{T}M(AA^{T})^{-1}M^{T}(Y_i - X_i \beta) \\
&= \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i^{T}Y_i - 2Y_i^{T}X_i \beta + \beta^{T}X_i^{T}X_i \beta) + \frac{1}{2 \sigma^4}(A^{-1}L^{T}Z_i^T{}(Y_i - X \beta))^{T}(A^{-1}L^{T}Z_i^T{}(Y_i - X \beta)) \\
&= \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i^{T}Y_i - 2Y_i^{T}X_i \beta + \beta^{T}X_i^{T}X_i \beta) \\
&+ \frac{1}{2 \sigma^4}(Z_i^{T}Y_i - Z_i^{T}X \beta)^{T}(A^{-1}L^{T})^{T}(A^{-1}L^{T})(Z_i^{T}Y_i - Z_i^{T}X \beta) \\
&= ((Z_i^{T}Y_i - Z_i^{T}X \beta)^{T}L)A^{T}A^{-1}L^{T}(Z_i^{T}Y_i - Z_i{T}Y_i - Z_i^{T}X \beta)
\end{align}

Which leaves us with the final form we will utilize to write an efficient function designed to calculate this value. 

#### Q2: Start-up code

Use the following template to define a type LmmObs that holds an LMM datum  (𝐲𝑖,𝐗𝑖,𝐙𝑖) .

In [15]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
# data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate arrays you may want to pre-allocate
    #storage_n :: Vector{T}
    storage_qq1 :: Matrix{T}
    storage_qq2 :: Matrix{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    ztz :: Matrix{T}
    yty :: T
    xty :: Vector{T}
    xtx :: Matrix{T}
    ztx :: Matrix{T}
    zty :: Vector{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}) where T <: AbstractFloat
    #storage_n = similar(y)
    ztz = transpose(Z) * Z
    det = similar(ztz)
    storage_qq1 = similar(ztz)
    storage_qq2 = similar(ztz)
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    yty = transpose(y) * y
    xty = transpose(X) * y
    xtx = transpose(X) * X
    ztx = transpose(Z) * X
    zty = transpose(Z) * y
    
    LmmObs(y, X, Z, storage_qq1, storage_qq2, storage_p, storage_q, ztz, yty, xty, xtx, ztx, zty)
end

LmmObs

Write a function, with interface:

`logl!(obs, β, L, σ²)`

that evaluates the log-likelihood of the  $i$ -th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition $\Sigma$=LL'. Make your code efficient in the $n_i >> q$  case. Think the intensive longitudinal measurement setting.

In [55]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    # TODO: compute and return the log-likelihood

    #sleep(1e-3) # wait 1 ms as if your code takes 1ms
    
    obs.storage_qq1 .= Matrix(I, q, q)
    
    mul!(obs.storage_qq2, transpose(L), obs.ztz) 
    # calculating L'Z'Z (making use of ztz which was precacluated)
    
    obs.storage_qq2 .= obs.storage_qq2 .* L # L'Z'Z L 
    #rmul!(obs.storage_qq2, LowerTriangular(L)) 
    # turns out rmul! is not more efficient ;_;
    
    axpy!(1/σ², obs.storage_qq2, obs.storage_qq1) # a*X + Y 
    # where a = 1/σ², X = M'M, 
    #Y = I, stores in third argument
    
    aat = cholesky!(Symmetric(obs.storage_qq1)) # AA'
    #a = potrf!(uplo = L, Symmetric(obs.storage_qq1))
    
    mul!(obs.storage_q, obs.ztx, β) 
    # Z'Xβ
    
    axpby!(1, obs.zty, -1, obs.storage_q) 
    #Z'Y - Z'Xβ from expansion of last term in log-likelihood
    
    mul!(obs.storage_p, obs.xtx, β) 
    # X'Xβ for faster multiplication in part of last term

    return -n//2 * log(2π) - n//2 * log(σ²) - 1//2 * logdet(aat) -
    1/(2 * (σ²)) * (obs.yty - 2 * transpose(β) * obs.xty +  transpose(β) * obs.storage_p) +
    1/(2 * (σ²)^2) * (dot(transpose(L) * obs.storage_q, aat \ transpose(L) * obs.storage_q)) 
    
end

logl! (generic function with 1 method)

#### Q3: Correctness

Compare your result (both accuracy and timing) to the Distributions.jl package using following data.

In [39]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.6949699666429332, 1.7723256696372407, 1.1065838446466518, 3.7291668118296073, 4.288899999400642, 2.8241842645202406  …  4.058027151891635, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.519302085567343, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [2.5057700244e-314 2.270241927e-314 5.0e-324; 2.544906727e-314 2.202366758e-314 NaN; 2.202366758e-314 2.202366758e-314 0.0007526874542236

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [40]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.179335805832

Check that your answer matches that from Distributions.jl

In [56]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3224.5007490251974

You will lose all 15 + 30 + 30 = 75 points if the following statement throws `AssertionError.`

In [14]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

#### Q4: Efficiency

Benchmarking your code and compare to the Distributions.jl function logpdf.

In [57]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 4148 samples with 1 evaluation.
 Range (min … max):  1.055 ms …  11.497 ms  ┊ GC (min … max): 0.00% … 89.43%
 Time  (median):     1.127 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.189 ms ± 235.524 μs  ┊ GC (mean ± σ):  0.21% ±  1.39%

   ▁██▆▆▅▄▄▃▃▃▃▃▂▂▁▁▁                                         ▁
  █████████████████████████▇▇█▇▆▇█▆▇▆▇▆▆▆▆▆▄▅▆▃▅▅▃▄▃▃▃▄▃▃▃▃▃▃ █
  1.05 ms      Histogram: log(frequency) by time      1.92 ms <

 Memory estimate: 31.52 KiB, allocs estimate: 3.

In [58]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.442 μs …  16.804 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.508 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.631 μs ± 360.549 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▇▆▅▃▃▁▁▁▁▁ ▁▁▁▁ ▁   ▁ ▁▁  ▁   ▁  ▁▁             ▁         ▂
  ████████████▇████▇██▆██▆██▆▆██▅███▆███▅▆█▇▆▄▇▇▆▄▄▅██▆▅▄▄▅▄▅ █
  1.44 μs      Histogram: log(frequency) by time      2.85 μs <

 Memory estimate: 400 bytes, allocs estimate: 6.

In [59]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

22.414986403130595

#### Q5: Memory

You want to avoid memory allocation in the "hot" function logl!. You will lose 1 point for each 1 KiB = 1024 bytes memory allocation. In other words, the points you get for this question is

In [60]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

29.609375